In [1]:
from pyabc.crystal.utils import non_dup_hnfs
from pyabc.io.vasp import read_vasp, write_vasp
from pyabc.crystal.structure import symbol2number as s2n
from pyabc.crystal.derive import ConfigurationGenerator, PermutationGroup

import os
import shutil

In [2]:
def _get_sites(l_atoms, ele, l_sub):
    ele_n = s2n(ele)
    l_sub_n = [s2n(sub_n) for sub_n in l_sub]
    sites = []
    for a in l_atoms:
        if a == ele_n:
            sites.append(tuple([a] + l_sub_n))
        else:
            sites.append(tuple([a]))
    return sites

In [3]:
symprec = 1e-3
comprec = 1e-3

dimension = 2
min_v = 7
max_v = 8

element = 'C'
substitutes = ['B', 'N']
comment = 'BxNxCy'

In [4]:
cell = read_vasp("./graphene.vasp")
cg = ConfigurationGenerator(cell, symprec)

In [5]:
if not os.path.exists('./BN-C-7to8'):
    os.mkdir('./BN-C-7to8')
else:
    shutil.rmtree('./BN-C-7to8')
    os.mkdir('./BN-C-7to8')

In [6]:
idx = 15840
for volume in range(min_v, max_v + 1):
    hnfs = non_dup_hnfs(cell, volume, dimension, symprec)
    for h in hnfs:
        hfpg = PermutationGroup(cell, h)
        perms = hfpg.get_symmetry_perms(symprec)
        
        supercell = cell.extend(h)
        sites = _get_sites(supercell.atoms.tolist(), element, substitutes)
        num_ele = len(supercell.atoms)
        for bn in range(1, num_ele//2):
            e_num = (num_ele-2*bn, bn, bn)
            for c, _ in cg._remove_redundant(supercell, sites,
                                              perms, e_num=e_num, remove_super=True):
                c = c.get_primitive_cell()
                filename = '{:s}_id{:d}'.format(comment, idx)
                idx += 1
                file = os.path.join('./BN-C-7to8', filename)
                write_vasp(c, file)